In [1]:
%run ./02_data_implementation.ipynb 

Vocab(n_tokens=146230| max_seq_len=79)
Vocab[87156]=балконных


In [2]:
%load_ext tensorboard
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%autoreload 2
from model_package.classifiers import LstmClassifier
import pytorch_lightning as pl 

from sklearn.utils import class_weight
from utils import save_model

from torch import nn
import torch 

In [4]:
from pathlib import Path
import os 

import warnings 
warnings.filterwarnings(action='ignore', category=pl.utilities.warnings.PossibleUserWarning)
warnings.filterwarnings(action='ignore', category=UserWarning)

In [5]:
## init model hyperparams: 
net = LstmClassifier(embedding_dim=300,
                     hidden_size=128,
                     num_layers=2,
                     bidirectional=True,
                     rnn_dropout=0.2,
                     vocab=vocab,
                     loss_fn=nn.CrossEntropyLoss()
                     )

In [7]:
logger = pl.loggers.TensorBoardLogger("logs")


In [12]:
## init training tools and training:
trainer = pl.Trainer(max_epochs=10, 
                     gradient_clip_algorithm='value',
                     accelerator='gpu',
                     logger=logger)

trainer.fit(net, 
            train_dataloaders=train_loader,
            val_dataloaders=test_loader)

In [2]:
%tensorboard --logdir logs/lightning_logs

In [14]:
trainer.validate(net, test_loader);

You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           Acc.            │     0.998854398727417     │
│           Auc.            │            1.0            │
│            F1             │    0.9988542199134827     │
│         val_loss          │    0.00468991557136178    │
└───────────────────────────┴───────────────────────────┘

In [15]:
trainer.validate(net, train_loader); 

You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           Acc.            │    0.9990261793136597     │
│           Auc.            │    0.9999993443489075     │
│            F1             │    0.9990246295928955     │
│         val_loss          │   0.004199056886136532    │
└───────────────────────────┴───────────────────────────┘

In [16]:
save_model(net, 'trained_models')